In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import requests
import datetime as dt
import requests
import json

In [2]:
now = dt.datetime.now()
date_today = str(now.day)+'-'+str(now.month)+'-'+str(now.year)
date_start = '02-01-2019'

# In case of scraping the web

In [10]:
#base_html = 'C://Users//bruno//Documents//DS//DataSets Grandes//Historicos Economicos//MEP Historico.html'
base_html = 'H://My Drive//Formacion//Data Science//Financial//FinancialPF//Web Sites//Turista Histórico.html'

### Fuente: https://www.ambito.com/contenidos/dolar-mep-historico.html

In [11]:
td_hist = pd.read_html(base_html) # this parses all the tables in webpages to a list
df_td = td_hist[0]
df_td.head() # Values are int without decimals, which is incorrect due to the real value's nature

,Fecha,Venta
0,05-11-2021,17399
1,04-11-2021,17391
2,03-11-2021,17388
3,02-11-2021,17384
4,01-11-2021,17383


In [12]:
def decimals(df, column='Value'):
    '''Converts integer value into float considering last 2 numbers as decimals. df parameter to specify df, column parameter to specify target column of df'''
    new_value_list = []
    for value in df[column]:
        ss1, ss2 = str(value)[:-2], str(value)[-2:] # Slicing the value as string to get the number separated 
        new = ss1+'.'+ss2 # Adding the . into the value
        new_value_list.append(float(new)) # Adding the new value to a list
    df[column] = pd.Series(new_value_list) # Converting it to a Series and inserting replacing the extracted values

In [13]:
df_td.columns = ["Date", 'Value']

In [14]:
decimals(df_td, 'Value')

In [15]:
df_td.head(5)

,Date,Value
0,05-11-2021,173.99
1,04-11-2021,173.91
2,03-11-2021,173.88
3,02-11-2021,173.84
4,01-11-2021,173.83


In [16]:
df_td.tail(5) # Checking last values with changes made

,Date,Value
466,02-01-2020,81.90
467,30-12-2019,81.89
468,27-12-2019,81.89
469,26-12-2019,81.89
470,23-12-2019,81.90


In [17]:
df_td.to_csv('H:\My Drive\Formacion\Data Science\Financial\FinancialPF\CSVs\TaxedDollar.csv', index=False)

# In case of using the web's API

In [3]:
# API Ámbito
request_url = 'https://mercados.ambito.com//dolarturista/historico-general/' #20-10-2021/20-11-2021'
request_url = request_url+date_start+'/'+date_today

In [4]:
response = requests.get(request_url)

In [5]:
response = response.text[1:]
response = response[:-1]
response = response.replace('],[', '];[')

In [6]:
(response).split(';')[10].replace('","', '";"') # Identifying tuples

'["08-11-2021";"174,21"]'

In [7]:
(response).split(';')[10].replace('","', '";"').replace('[','').replace(']','').replace('"','').split(';') # Splitting values

['08-11-2021', '174,21']

In [8]:
prices = []
for i in range (0,1000):
    try:
        prices.append(((response).split(';')[i].replace('","', '";"').replace('[','').replace(']','').replace('"','').replace(',','.').split(';')))
    except:
        pass

In [9]:
TaxedUSD = pd.DataFrame(prices[1:], columns=prices[0])
TaxedUSD.Venta = TaxedUSD.Venta.astype('float')

In [10]:
TaxedUSD.drop_duplicates(subset=['Fecha'], keep='first', inplace=True)

In [11]:
path = 'H:\My Drive\Formacion\Data Science\Financial\FinancialPF\CSVs\\'
TaxedUSD.to_csv(path+'TaxedUSD_API.csv', index=False)